# Geotail PWI SFA plot -- 2025/8/15

In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import sys

# <<< if needed: spacepy install >>>
# !pip install spacepy

In [ ]:
# <DATA selection flag>
mode_PWIdata      = 0       # 0: CDF     1: ASCII
# <Noise Reject flag
mode_noise_reject = 1       # 0: n/a     1: Noise Reject

sys.path.append("./lib/")
if mode_PWIdata == 0:
    import gtl_pwi_sfa_cdf_lib as gtl_pwi_sfa
else:
    import gtl_pwi_sfa_asc_lib as gtl_pwi_sfa
import gtl_kprm_fx_lib as gtl_kprm_fx

# parameter set -- to be modified

In [ ]:
Epoch_min = '1992-09-01 00:00:00';  t_min0 = datetime.datetime.strptime(Epoch_min, "%Y-%m-%d %H:%M:%S")     # Start time
Epoch_max = '1992-09-30 23:59:59';  t_max0 = datetime.datetime.strptime(Epoch_max, "%Y-%m-%d %H:%M:%S")     # End   time

p_min0    = -190.;    p_max0 = -100.                    # [range]       E (dB V2/m2/Hz)
f_min0    = 12500;    f_max0 = 800000.                  # [range:       frequency r(Hz)   (1562.5 - 8000000 Hz)

mode_freq = 0                                           # [frequency]   0: linear   1: log
mode_gap  = 1                                           # [gap]         0: n/a      1: NAN in time gap
mode_stack= 0                                           # [stack]       0: normal   1: Stack plot only
mode_check= 0                                           # [check]       0: plot     1: no plot
mode_plot = 1                                           # [plot]        0: none     1: plot dump

work_dir  = '/Users/user/0-python/GTL_python/ql/'       # Plot dump folder
data_dir  = '/Users/user/D-data/data-Geotail/'          # Data folder

# Data Read

### SFA

In [ ]:
sfa  = gtl_pwi_sfa.read_sfa_multi(Epoch_min, Epoch_max, mode_gap, mode_check, data_dir)
if sfa.num == 0:
    print("!!!!! SFA: NO DATA !!!!!")
else:
    print('[Total]', sfa.num, '     (', sfa.epoch[0], '-', sfa.epoch[-1],  ')')


In [ ]:
# Data check
SFA_E  = sfa.sfae_high.transpose()
n0 = 0;  n1 = sfa.num//2;   n2 = sfa.num-1
print(' [Start]', sfa.epoch[n0],  f'  E[dB V2/m2/Hz]  Band-3 ({sfa.sfae_high[n0][0]:.1f} - {sfa.sfae_high[n0][127]:.1f})', f'  Band-4 ({sfa.sfae_high[n0][128]:.1f} - {sfa.sfae_high[n0][255]:.1f})', f'  Band-5 ({sfa.sfae_high[n0][256]:.1f} - {sfa.sfae_high[n0][383]:.1f})')
print('   [Mid]', sfa.epoch[n1],  f'  E[dB V2/m2/Hz]  Band-3 ({sfa.sfae_high[n1][0]:.1f} - {sfa.sfae_high[n1][127]:.1f})', f'  Band-4 ({sfa.sfae_high[n1][128]:.1f} - {sfa.sfae_high[n1][255]:.1f})', f'  Band-5 ({sfa.sfae_high[n1][256]:.1f} - {sfa.sfae_high[n1][383]:.1f})')
print('   [End]', sfa.epoch[n2],  f'  E[dB V2/m2/Hz]  Band-3 ({sfa.sfae_high[n2][0]:.1f} - {sfa.sfae_high[n2][127]:.1f})', f'  Band-4 ({sfa.sfae_high[n2][128]:.1f} - {sfa.sfae_high[n2][255]:.1f})', f'  Band-5 ({sfa.sfae_high[n2][256]:.1f} - {sfa.sfae_high[n2][383]:.1f})')

### ORBIT

In [ ]:
orbit  = gtl_kprm_fx.read_orbit_multi(Epoch_min, Epoch_max, mode_gap, mode_check, data_dir)
if orbit.num == 0:
    print("!!!!! ORBIT: NO DATA !!!!!")
else:
    print('[Total]', orbit.num, '     (', orbit.epoch[0], '-', orbit.epoch[-1],  ')')

In [ ]:
# Data check
orbit_SM  = orbit.orbit_SM.transpose()
m0 = 0;  m1 = orbit.num//2;   m2 = orbit.num-1
print(' [Start]', orbit.epoch[m0],  f'  SM[Re] ({orbit.orbit_SM[m0][0]:+6.1f}, {orbit.orbit_SM[m0][1]:+6.1f}, {orbit.orbit_SM[m0][2]:+6.1f})', f'   R[Re] {orbit.R[m0]:5.1f}', f'   MLAT[deg] {orbit.MLAT[m0]:+5.1f}', f'   MLT[h] {orbit.R[m0]:4.1f}', f'   Tilt[deg] {orbit.tilt[m0]:4.1f}')
print('   [Mid]', orbit.epoch[m1],  f'  SM[Re] ({orbit.orbit_SM[m1][0]:+6.1f}, {orbit.orbit_SM[m1][1]:+6.1f}, {orbit.orbit_SM[m1][2]:+6.1f})', f'   R[Re] {orbit.R[m1]:5.1f}', f'   MLAT[deg] {orbit.MLAT[m1]:+5.1f}', f'   MLT[h] {orbit.R[m1]:4.1f}', f'   Tilt[deg] {orbit.tilt[m1]:4.1f}')
print('   [End]', orbit.epoch[m2],  f'  SM[Re] ({orbit.orbit_SM[m2][0]:+6.1f}, {orbit.orbit_SM[m2][1]:+6.1f}, {orbit.orbit_SM[m2][2]:+6.1f})', f'   R[Re] {orbit.R[m2]:5.1f}', f'   MLAT[deg] {orbit.MLAT[m2]:+5.1f}', f'   MLT[h] {orbit.R[m2]:4.1f}', f'   Tilt[deg] {orbit.tilt[m2]:4.1f}')

# Plot

### Noise rejection

In [ ]:
if mode_noise_reject > 0:   # 0: n/a     1: Noise Reject
    print("!!! Noise rejection in Band-4 and Band-5 in E-field !!!")
    for i in range (sfa.num):
        sfa.sfae_high[i][128   +  78] = (sfa.sfae_high[i][128   +  77] * 2 + sfa.sfae_high[i][128   +  80]    ) / 3
        sfa.sfae_high[i][128   +  79] = (sfa.sfae_high[i][128   +  77]     + sfa.sfae_high[i][128   +  80] * 2) / 3
        #
        sfa.sfae_high[i][128*2 +   3] = (sfa.sfae_high[i][128*2 +   2]     + sfa.sfae_high[i][128*2 +   4]    ) / 2
        #
        sfa.sfae_high[i][128*2 +  23] = (sfa.sfae_high[i][128*2 +  22] * 2 + sfa.sfae_high[i][128*2 +  25]    ) / 3
        sfa.sfae_high[i][128*2 +  24] = (sfa.sfae_high[i][128*2 +  22]     + sfa.sfae_high[i][128*2 +  25] * 2) / 3
        #
        sfa.sfae_high[i][128*2 +  79] = (sfa.sfae_high[i][128*2 +  78]     + sfa.sfae_high[i][128*2 +  80]    ) / 2
        #
        sfa.sfae_high[i][128*2 +  87] = (sfa.sfae_high[i][128*2 +  86] * 2 + sfa.sfae_high[i][128*2 +  89]    ) / 3
        sfa.sfae_high[i][128*2 +  88] = (sfa.sfae_high[i][128*2 +  86]     + sfa.sfae_high[i][128*2 +  89] * 2) / 3
        #
        sfa.sfae_high[i][128*2 +  96] = (sfa.sfae_high[i][128*2 +  95] * 3 + sfa.sfae_high[i][128*2 +  99]    ) / 4
        sfa.sfae_high[i][128*2 +  97] = (sfa.sfae_high[i][128*2 +  95] * 2 + sfa.sfae_high[i][128*2 +  99] * 2) / 4
        sfa.sfae_high[i][128*2 +  98] = (sfa.sfae_high[i][128*2 +  95]     + sfa.sfae_high[i][128*2 +  99] * 3) / 4
        #
        sfa.sfae_high[i][128*2 + 109] = (sfa.sfae_high[i][128*2 + 108]     + sfa.sfae_high[i][128*2 + 110]    ) / 2
sfa_med = np.nanmedian(sfa.sfae_high, axis=0)

### Spectrum

In [ ]:
if mode_stack==0 and mode_check==0:
    n0 = 0;  n1 = sfa.num//2;   n2 = sfa.num-1
    p_min = p_min0;  p_max = p_max0
    f_min = f_min0;  f_max = f_max0
    date1 = sfa.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
    date2 = sfa.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
    date3 = sfa.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')

    fig = plt.figure(figsize=(14, 11));  ax1 = fig.add_subplot(1, 1, 1)
    ax1.plot(sfa.freq_e, sfa_med,   '-y',  linewidth=3.0, label='median')
    ax1.plot(sfa.freq_e, sfa.sfae_high[n0], '-r',  linewidth=1.0, label=date1)
    ax1.plot(sfa.freq_e, sfa.sfae_high[n1], '-g',  linewidth=1.0, label=date2)
    ax1.plot(sfa.freq_e, sfa.sfae_high[n2], '-b',  linewidth=1.0, label=date3)
    ax1.legend(loc='upper right', fontsize=8)

    # X-axis
    ax1.set_xlabel('frequency [Hz]')
    xlim=[f_min, f_max];  ax1.set_xlim(xlim)
    title_date = "[Geotail PWI SFA]  " + date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title(title_date)

    # Y-axis
    ax1.set_ylabel('E [dB V2/m2/Hz]');   
    ylim=[p_min, p_max];  ax1.set_ylim(ylim)
    if mode_freq == 1:
        ax1.set_xscale('log')

    fig.subplots_adjust(hspace=0.1);  fig.show
    if mode_plot == 1:
        png_filename = work_dir + 'GTL-PWI-SFA-spec_' + sfa.epoch[n0].strftime('%Y%m%d%H%M-') + sfa.epoch[n2].strftime('%Y%m%d%H%M') + '.png'
        fig.savefig(png_filename);  print(png_filename)

### FT diagram

In [ ]:
if mode_stack==0 and mode_check==0:
    n0 = 0;  n2 = sfa.num-1
    p_min = p_min0;  p_max = p_max0
    f_min = f_min0;  f_max = f_max0
    date1 = sfa.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
    date3 = sfa.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')

    fig2d = plt.figure(figsize=[16, 11])
    ax1   = fig2d.add_subplot(1, 1, 1)
    p1    = ax1.pcolormesh(sfa.epoch, sfa.freq_e, SFA_E, vmin=p_min, vmax=p_max, cmap='jet')
    pp1   = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label('dB V2/m2/Hz')

    # X-axis
    if t_min0 == 0:
        E_min = '2000-08-19 02:55:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        E_max = '2000-08-19 02:57:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        xlim=[sfa.epoch[0], sfa.epoch[-1]]
    else:
        xlim=[t_min0, t_max0]
    ax1.set_xlim(xlim)
    title_date = "[Geotail PWI SFA]  " + date1 + "  -  " + date3;  ax1.set_title(title_date)

    # Y-axis
    ax1.set_ylabel('frequecy [Hz]');  ax1.set_ylim(f_min, f_max);  
    if mode_freq == 1:
        ax1.set_yscale('log')

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if mode_plot == 1:
        png_filename = work_dir + 'GTL-PWI-SFA-ft_' + xlim[0].strftime('%Y%m%d%H%M-') + xlim[1].strftime('%Y%m%d%H%M') + '.png'
        fig2d.savefig(png_filename);  print(png_filename)
        plt.subplots_adjust(hspace=0.02);  plt.show()

### Orbit

In [ ]:
if mode_stack==0 and mode_check==0:
    fig = plt.figure(figsize=(14, 11))
    ax1 = fig.add_subplot(3, 1, 1);  ax2 = fig.add_subplot(3, 1, 2);  ax3 = fig.add_subplot(3, 1, 3)
    ax1.plot(orbit.epoch, orbit.R,           '-y', linewidth=3, label='R')
    ax1.plot(orbit.epoch, orbit_SM[0], '-r', linewidth=1, label='X_SM')
    ax1.plot(orbit.epoch, orbit_SM[1], '-g', linewidth=1, label='Y_SM')
    ax1.plot(orbit.epoch, orbit_SM[2], '-b', linewidth=1, label='Z_SM')
    ax2.plot(orbit.epoch, orbit.MLAT,        '-k', linewidth=1, label='MLAT')
    ax2.plot(orbit.epoch, orbit.tilt,        '-r', linewidth=1, label='Tilt angle')
    ax3.plot(orbit.epoch, orbit.MLT,         '-k', linewidth=1, label='MLT')
    ax1.set_ylabel('SM [Re]');  ax2.set_ylabel('MLAT & Tilt Angle [deg]');  ax3.set_ylabel('MLT [h]')
    ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)

    # X-axis
    if t_min0 == 0:
        E_min = '2000-01-19 02:55:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        E_max = '2000-01-19 02:57:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        xlim=[orbit.epoch[0], orbit.epoch[-1]]
    else:
        xlim=[t_min0, t_max0]
    ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim)
    title_label = '[Geotail Orbit] ' + xlim[0].strftime('%Y%m%d %H%M - ') + xlim[1].strftime('%Y%m%d %H%M');  ax1.set_title(title_label)

    # Y-axis
    ylim=[0, 24];  ax3.set_ylim(ylim);  ax3.set_yticks([0,3,6,9,12,15,18,21,24])

    fig.subplots_adjust(hspace=0.12);  fig.show
    if mode_plot == 1:
        png_filename = work_dir + 'GTL-ORB_' + xlim[0].strftime('%Y%m%d%H%M-') + xlim[1].strftime('%Y%m%d%H%M') + '.png'
        fig.savefig(png_filename);  print(png_filename)

### FT diagram + Orbit

In [ ]:
if mode_check==0:
    n0 = 0;  n2 = sfa.num-1
    p_min = p_min0;  p_max = p_max0
    f_min = f_min0;  f_max = f_max0
    date1 = sfa.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
    date3 = sfa.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')

    fig2d = plt.figure(figsize=[16, 11])
    ax1   = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3);  ax4 = fig2d.add_subplot(4, 1, 4)
    p1    = ax1.pcolormesh(sfa.epoch, sfa.freq_e, SFA_E, vmin=p_min, vmax=p_max, cmap='jet')
    pp1   = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label('dB V2/m2/Hz')
    p2    = ax2.plot(orbit.epoch, orbit.R,    '-k', linewidth=1, label='R')
    p3    = ax3.plot(orbit.epoch, orbit.MLAT, '-k', linewidth=1, label='MLAT')
    p3    = ax3.plot(orbit.epoch, orbit.tilt, '-r', linewidth=1, label='Tilt angle')
    p4    = ax4.plot(orbit.epoch, orbit.MLT,  '-k', linewidth=1, label='MLT')
    pp1   = fig2d.colorbar(p1, ax=ax2, orientation="vertical");  pp1.set_label('dB V2/m2/Hz')
    pp1   = fig2d.colorbar(p1, ax=ax3, orientation="vertical");  pp1.set_label('dB V2/m2/Hz')
    pp1   = fig2d.colorbar(p1, ax=ax4, orientation="vertical");  pp1.set_label('dB V2/m2/Hz')
    ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);  ax4.legend(loc='upper right', fontsize=8)

    # X-axis
    if t_min0 == 0:
        E_min = '2000-08-19 02:55:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        E_max = '2000-08-19 02:57:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        xlim=[sfa.epoch[0], sfa.epoch[-1]]
    else:
        xlim=[t_min0, t_max0]
    ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)
    title_date = "[Geotail PWI SFA]  " + date1 + "  -  " + date3;  ax1.set_title(title_date)

    # Y-axis
    ax1.set_ylabel('frequecy [Hz]');  ax2.set_ylabel('R [Re]');  ax3.set_ylabel('MLAT & Tilt Angle [deg]');  ax4.set_ylabel('MLT [h]')
    ax1.set_ylim(f_min, f_max);  
    if mode_freq == 1:
        ax1.set_yscale('log')
    ylim=[0, 24];  ax4.set_ylim(ylim);  ax4.set_yticks([0,3,6,9,12,15,18,21,24])

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if mode_plot == 1:
        png_filename = work_dir + 'GTL-PWI-SFA-ORB_' + xlim[0].strftime('%Y%m%d%H%M-') + xlim[1].strftime('%Y%m%d%H%M') + '.png'
        fig2d.savefig(png_filename);  print(png_filename)